In [ ]:
import hoomd
import numpy as np

import gsd.hoomd

from monk import nb
from monk import prep
from monk import pair
import freud

In [ ]:
class Spinner2D(hoomd.md.force.Custom):
    def __init__(self, torque):
        super().__init__(aniso=True)

        self._torque = torque

    def set_forces(self, timestep):
        with self.cpu_local_force_arrays as arrays:
            # arrays.force[:] = -5
            arrays.torque[:,2] = self._torque
            # arrays.potential_energy[:] = 27
            # arrays.virial[:] = np.arange(6)[None, :]

In [ ]:

def gen_highT_state():
    seed = 1000
    cpu = hoomd.device.auto_select()
    print(cpu)
    sim = hoomd.Simulation(cpu, seed=seed)
    N = 256
    rng = prep.init_rng(seed + 1)
    L = prep.len_from_phi(N, 1.1, dim=2)
    snap = prep.approx_euclidean_snapshot(N, L, rng, dim=2, ratios=[100], diams=[1.0], particle_types=['A'])
    # print(snap.particles.moment_inertia)
    snap.particles.moment_inertia = N*[[1.0, 1.0, 1.0]]
    snap.particles.orientation = N*[np.sqrt(1/2), 0.0, np.sqrt(1/2), 0.0]
    snap.particles.velocity = np.random.random((N, 3))
    snap.particles.velocity[:,2] = 0.0

    # assert isinstance(snap, gsd.hoomd.Snapshot)

    sim.create_state_from_snapshot(snap)

    integrator = hoomd.md.Integrator(dt=0.0025, integrate_rotational_dof=True)
    tree = hoomd.md.nlist.Tree(0.3)
    # gb = hoomd.md.pair.aniso.GayBerne(tree, default_r_cut=2.5)
    gb = hoomd.md.pair.LJ(tree, default_r_cut=2.5)
    gb.params[('A', 'A')] = dict(epsilon=1.0, sigma=1.0)
    gb.r_cut[('A', 'A')] = 2.5
    
    spin = Spinner2D(1.0)
    nvt = hoomd.md.methods.NVE(hoomd.filter.All())
    integrator.forces = [gb, spin]
    integrator.methods = [nvt]
    # plane = hoomd.md.manifold.Plane()
    # constraint = hoomd.md.constrain.Constraint()
    # integrator.constraints = [plane]

    sim.always_compute_pressure = True
    thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
        filter=hoomd.filter.All())
    sim.operations.computes.append(thermodynamic_properties)

    sim.operations.integrator = integrator

    sim.run(0)

    # nvt.thermalize_thermostat_dof()

    # sim.run(100)

    # nvt.thermalize_thermostat_dof()

    # sim.run(40_000)

    print(thermodynamic_properties.pressure)

    logger = hoomd.logging.Logger()
    logger.add(gb, quantities=['forces'])
    logger.add(spin, quantities=['torques'])

    writer = hoomd.write.GSD(trigger=hoomd.trigger.Periodic(1), filename="trial2d.gsd", mode="wb", filter=hoomd.filter.All(), dynamic=["property", "momentum", "attribute"], log=logger)
    sim.operations.writers.append(writer)

    # print(sim.state.get_snapshot().particles.angmom)

    for i in range(5):

        sim.run(401, True)

        # print(gb.forces)

        break

        # nvt.kT = float(nvt.kT.value) / 1.5

        print(i, thermodynamic_properties.pressure)
        # print(sim.state.get_snapshot().particles.angmom)

    hoomd.write.GSD.write(sim.state, "init-state2d.gsd")

    return snap
    

In [ ]:
snap = gen_highT_state()

In [ ]:
snap.configuration.dimensions

In [ ]:
snap.particles.orientation

In [ ]:
snap.particles.angmom

In [ ]:
traj = gsd.hoomd.open("trial2d.gsd")
dt = 0.0025

In [ ]:
def quat_to_axis_angle(quat):
    s = quat[0]
    v = quat[1:]
    theta = np.arccos(s)*2.0
    v /= np.sin(theta/2.0)
    return np.concatenate([[theta], v])

In [ ]:
i = 0

orients = []
angmoms = []
torques = []

for idx in range(len(traj)):
    snap = traj[idx]
    angmom = snap.particles.angmom[i]
    # angmom_mag = np.linalg.norm(angmom)
    # angmom_normed = quat_to_axis_angle(angmom/angmom_mag)
    orient = snap.particles.orientation[i]
    # force = snap.log["particles/md/pair/aniso/GayBerne/forces"][i]
    torque = snap.log["particles/__main__/Spinner2D/torques"][i]
    orients.append(orient)
    angmoms.append(angmom)
    torques.append(torque)
    # print(angmom_normed[1:].dot(orient[1:]))
    # print(angmom_normed[1:].dot(torque))
    # print(angmom_mag, angmom_normed, orient, torque)

In [ ]:
list(snap.log.keys())

In [ ]:
angmoms[4]

In [ ]:
snap.particles.moment_inertia[i]

In [ ]:

def conv(q):
    return np.array([q[0], -q[1], -q[2], -q[3]], dtype=np.float32)

def quat(v):
    return np.array([0.0, v[0], v[1], v[2]], dtype=np.float32)

def qmult(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

def rotate(q, v):
    return qmult(qmult(q, quat(v)), conv(q))[1:]

In [ ]:
quat(torques[0]), orients[0]

In [ ]:
snap.particles.moment_inertia[i]

In [ ]:
j = 1
orients[j], angmoms[j]

In [ ]:
idx = 110
angmom_0 = qmult(conv(orients[idx]), angmoms[idx])[1:]/2.0
angmom_1 = qmult(conv(orients[idx + 1]), angmoms[idx + 1])[1:]/2.0
torque_0 = torques[idx]
# torque_0[:2] = 0.0
update = dt * qmult(qmult(conv(orients[idx]), quat(torque_0)), orients[idx])[1:]
# angmom_0 = qmult(angmoms[idx], conv(orients[idx]))[1:]/2.0
# angmom_1 = qmult(angmoms[idx + 1], conv(orients[idx + 1]))[1:]/2.0
# update = dt * torques[idx]
# update[1:] = 0.0
print(orients[idx], update, torques[idx])
angmom_0, angmom_0 + update, angmom_1

In [ ]:
qmult(qmult(orients[idx], quat(angmom_0)), conv(orients[idx]))

In [ ]:
qmult(angmoms[idx], conv(orients[idx]))/2.0

In [ ]:
idx = 110
angmom_0 = qmult(angmoms[idx], conv(orients[idx]))[1:]/2.0
angmom_1 = qmult(angmoms[idx + 1], conv(orients[idx + 1]))[1:]/2.0
torque_0 = torques[idx]
# torque_0[:2] = 0.0
update = dt * torque_0
# angmom_0 = qmult(angmoms[idx], conv(orients[idx]))[1:]/2.0
# angmom_1 = qmult(angmoms[idx + 1], conv(orients[idx + 1]))[1:]/2.0
# update = dt * torques[idx]
# update[1:] = 0.0
print(orients[idx], update, torques[idx])
angmom_0, angmom_0 + update, angmom_1

In [ ]:
quat_to_axis_angle(orients[idx])

In [ ]:
(angmoms[1][1:] - angmoms[0][1:])/torques[0]

In [ ]:
(qmult(conv(orients[0]), angmoms[0])[1:]/2 - qmult(conv(orients[1]), angmoms[1])[1:]/2)/torques[0]

In [ ]:
torques[0]